In [1]:
## importacion de la data 
import numpy as np
import pandas as pd

data = pd.read_csv('./data_set/student-mat.csv', sep=';')

def cambioTextNum(df, nomCol, tipo):
    df[nomCol] = df[nomCol].astype(tipo)
    return df[nomCol]

def camTextBi(df, nomCol, busqueda, camuno, camdos):
    df[nomCol] = np.where(df[nomCol] == busqueda, camuno, camdos)
    return df[nomCol]

def convDummies(df, nomCol):
    dummies = pd.get_dummies(df[nomCol])
    data = pd.concat([df, dummies], axis=1)
    data.drop(columns=[nomCol], inplace=True)
    return data

def datos_NA(df):
    if isinstance(df, pd.DataFrame):
        total_na = df.isna().sum().sum()
        print("Dimensiones : %d filas, %d columnas" %
              (df.shape[0], df.shape[1]))
        print("Total Valores NA : %d " % (total_na))
        print("%38s %10s     %10s %10s" %
              ("Nombre Columna", "Tipo de Dato", "#Distintos", "Valores NA"))
        col_name = df.columns
        dtyp = df.dtypes
        uniq = df.nunique()
        na_val = df.isna().sum()
        for i in range(len(df.columns)):
            print("%38s %10s   %10s %10s" %
                  (col_name[i], dtyp[i], uniq[i], na_val[i]))

    else:
        print("Se esperaba dataframe %15s" % (type(df)))

data.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [2]:
datos_NA(data)

Dimensiones : 395 filas, 33 columnas
Total Valores NA : 0 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                                school     object            2          0
                                   sex     object            2          0
                                   age      int64            8          0
                               address     object            2          0
                               famsize     object            2          0
                               Pstatus     object            2          0
                                  Medu      int64            5          0
                                  Fedu      int64            5          0
                                  Mjob     object            5          0
                                  Fjob     object            5          0
                                reason     object            4          0
                              guardian     object

In [3]:
data['sex'] = camTextBi(data,'sex','F',0,1)
data['school'] = camTextBi(data,'school','GP',0,1)
data['address'] = camTextBi(data,'address','U',0,1)
data['address'] = camTextBi(data,'address','U',0,1)
data['famsize'] = camTextBi(data,'famsize','GT3',0,1)
data['Pstatus'] = camTextBi(data,'Pstatus','A',0,1)
data['schoolsup'] = camTextBi(data,'schoolsup','yes',0,1)
data['famsup'] = camTextBi(data,'famsup','yes',0,1)
data['paid'] = camTextBi(data,'paid','yes',0,1)
data['activities'] = camTextBi(data,'activities','yes',0,1)
data['nursery'] = camTextBi(data,'nursery','yes',0,1)
data['higher'] = camTextBi(data,'higher','yes',0,1)
data['internet'] = camTextBi(data,'internet','yes',0,1)
data['romantic'] = camTextBi(data,'romantic','yes',0,1)

In [4]:
datos_NA(data)

Dimensiones : 395 filas, 33 columnas
Total Valores NA : 0 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                                school      int64            2          0
                                   sex      int64            2          0
                                   age      int64            8          0
                               address      int64            1          0
                               famsize      int64            2          0
                               Pstatus      int64            2          0
                                  Medu      int64            5          0
                                  Fedu      int64            5          0
                                  Mjob     object            5          0
                                  Fjob     object            5          0
                                reason     object            4          0
                              guardian     object

In [16]:
DataDumi_1 = convDummies(data,'Mjob')
DataDumi_2 = convDummies(DataDumi_1,'Fjob')
DataDumi_3 = convDummies(DataDumi_2,'reason')
DataTestNor = convDummies(DataDumi_3,'guardian')

In [17]:
datos_NA(DataTestNor)

Dimensiones : 395 filas, 46 columnas
Total Valores NA : 0 
                        Nombre Columna Tipo de Dato     #Distintos Valores NA
                                school      int64            2          0
                                   sex      int64            2          0
                                   age      int64            8          0
                               address      int64            1          0
                               famsize      int64            2          0
                               Pstatus      int64            2          0
                                  Medu      int64            5          0
                                  Fedu      int64            5          0
                            traveltime      int64            4          0
                             studytime      int64            4          0
                              failures      int64            4          0
                             schoolsup      int64

In [18]:
from scipy import stats
def testShapiroWilk(df):
        valoresP = []
        concepto = []
        variable = []
        for column in df:
            k2, p_value = stats.shapiro(df[column].values)
            valoresP.append(p_value)
            variable.append(column)
            if (p_value < 0.05):
                concepto.append('No es una variable Normal')
            else:
                concepto.append('Es una variable Normal')
        dfShapiro = pd.DataFrame(
            {'Variable': variable, 'Valores P': valoresP, 'Concepto': concepto})
        return dfShapiro


In [26]:

data = testShapiroWilk(DataTestNor)
data

/opt/homebrew/lib/python3.11/site-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


,Variable,Valores P,Concepto
0,school,1.151195e-34,No es una variable Normal
1,sex,4.010956e-28,No es una variable Normal
2,age,1.587842e-14,No es una variable Normal
3,address,1.000000e+00,Es una variable Normal
4,famsize,4.301020e-30,No es una variable Normal
5,Pstatus,3.964336e-35,No es una variable Normal
6,Medu,2.815386e-18,No es una variable Normal
7,Fedu,2.706857e-17,No es una variable Normal
8,traveltime,2.310120e-27,No es una variable Normal
9,studytime,6.547986e-20,No es una variable Normal


In [32]:
from sklearn.preprocessing import MinMaxScaler
def normaData(df):
    valores = df.values  
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler = scaler.fit(valores)

    pd.DataFrame(np.vstack((scaler.data_min_, scaler.data_max_)),
             index=['Min', 'Max'],
             columns=df.columns)

    normalizados = scaler.transform(valores)
    df_norm = pd.DataFrame(normalizados,
                       index=df.index,
                       columns=df.columns)
    return df_norm

In [33]:
dataNormalizada = normaData(DataTestNor)
dataf = testShapiroWilk(dataNormalizada)
dataf

/opt/homebrew/lib/python3.11/site-packages/scipy/stats/_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


,Variable,Valores P,Concepto
0,school,1.151195e-34,No es una variable Normal
1,sex,4.010956e-28,No es una variable Normal
2,age,1.587842e-14,No es una variable Normal
3,address,1.000000e+00,Es una variable Normal
4,famsize,4.301020e-30,No es una variable Normal
5,Pstatus,3.964336e-35,No es una variable Normal
6,Medu,2.815386e-18,No es una variable Normal
7,Fedu,2.706857e-17,No es una variable Normal
8,traveltime,2.310120e-27,No es una variable Normal
9,studytime,6.547986e-20,No es una variable Normal


In [36]:
DataTestNor.describe().T

,count,mean,std,min,25%,50%,75%,max
school,395.0,0.116456,0.321177,0.0,0.0,0.0,0.0,1.0
sex,395.0,0.473418,0.499926,0.0,0.0,0.0,1.0,1.0
age,395.0,16.696203,1.276043,15.0,16.0,17.0,18.0,22.0
address,395.0,1.000000,0.000000,1.0,1.0,1.0,1.0,1.0
famsize,395.0,0.288608,0.453690,0.0,0.0,0.0,1.0,1.0
Pstatus,395.0,0.896203,0.305384,0.0,1.0,1.0,1.0,1.0
Medu,395.0,2.749367,1.094735,0.0,2.0,3.0,4.0,4.0
Fedu,395.0,2.521519,1.088201,0.0,2.0,2.0,3.0,4.0
traveltime,395.0,1.448101,0.697505,1.0,1.0,1.0,2.0,4.0
studytime,395.0,2.035443,0.839240,1.0,1.0,2.0,2.0,4.0
